# NutriGenius: Article Recommender - Exploratory Data Analysis

Notebook ini melakukan exploratory data analysis pada dataset artikel nutrisi untuk membangun sistem rekomendasi artikel dalam aplikasi NutriGenius.

## Daftar Isi
1. [Pendahuluan](#pendahuluan)
2. [Setup Dataset](#setup-dataset)
3. [Eksplorasi Dataset](#eksplorasi-dataset)
4. [Analisis Teks](#analisis-teks)
5. [Visualisasi Data](#visualisasi-data)
6. [Kesimpulan](#kesimpulan)

## 1. Pendahuluan

Notebook ini mengeksplorasi dataset artikel nutrisi untuk mengembangkan komponen rekomendasi artikel pada aplikasi NutriGenius. Sistem rekomendasi akan memberikan artikel nutrisi yang relevan berdasarkan:
- Profil pengguna (usia, gender) dari komponen deteksi wajah
- Makanan yang dikonsumsi dari komponen deteksi objek makanan
- Informasi teks dari komponen pengenalan teks (OCR)

Fokus eksplorasi ini mencakup:
- Pemahaman distribusi topik dan konten artikel
- Analisis kata kunci dan terminologi nutrisi
- Persiapan data untuk sistem rekomendasi

## 2. Setup Dataset

Untuk sistem rekomendasi artikel, kita membutuhkan dataset artikel nutrisi dan dataset tanya jawab (Q&A) nutrisi. Kita akan membuat dataset sintetis khusus untuk proof-of-concept ini.

In [ ]:
import os
import sys
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
import logging
import urllib.request

# Konfigurasi logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Path konfigurasi
BASE_DIR = os.path.abspath(os.path.join(os.path.dirname("__file__"), '..', '..'))
DATA_DIR = os.path.join(BASE_DIR, "data")
RAW_DATA_DIR = os.path.join(DATA_DIR, "raw")
ARTICLES_DATA_DIR = os.path.join(RAW_DATA_DIR, "articles")
PROCESSED_DATA_DIR = os.path.join(DATA_DIR, "processed", "articles")

# Buat direktori jika belum ada
for directory in [DATA_DIR, RAW_DATA_DIR, ARTICLES_DATA_DIR, PROCESSED_DATA_DIR]:
    os.makedirs(directory, exist_ok=True)

print(f"Base directory: {BASE_DIR}")
print(f"Data directory: {DATA_DIR}")
print(f"Articles data will be stored in: {ARTICLES_DATA_DIR}")

# Download NLTK resources jika belum ada
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

### 2.1 Membuat Dataset Artikel Nutrisi

Kita akan membuat dataset artikel nutrisi sintetis dan dataset tanya jawab (Q&A) nutrisi untuk sistem rekomendasi.

In [ ]:
def download_with_progress(url, dest_path):
    """Download file dengan progress bar"""
    try:
        with tqdm(unit='B', unit_scale=True, miniters=1, desc=url.split('/')[-1]) as t:
            urllib.request.urlretrieve(
                url, 
                dest_path, 
                reporthook=lambda b, bsize, tsize: t.update(bsize if tsize == -1 else min(bsize, tsize-t.n))
            )
        return True
    except Exception as e:
        logger.error(f"Gagal mengunduh {url}: {e}")
        return False

def setup_articles_dataset(create_synthetic=True, download_external=False):
    """
    Menyiapkan dataset artikel nutrisi
    
    Args:
        create_synthetic: Jika True, buat dataset artikel sintetis
        download_external: Jika True, coba unduh dataset eksternal (jika tersedia)
        
    Returns:
        bool: True jika berhasil, False jika gagal
    """
    # Periksa apakah dataset sudah ada
    articles_json_path = os.path.join(ARTICLES_DATA_DIR, "articles_data.json")
    articles_csv_path = os.path.join(ARTICLES_DATA_DIR, "nutrition_qa.csv")
    
    if os.path.exists(articles_json_path) and os.path.exists(articles_csv_path):
        logger.info("Dataset artikel nutrisi sudah ada.")
        return True
    
    success = True
    
    # Coba unduh dataset eksternal jika diminta
    if download_external:
        EXTERNAL_ARTICLES_URL = "https://github.com/nutrigenius-samples/nutrition-articles/raw/main/nutrition_articles.zip"
        EXTERNAL_ZIP = os.path.join(DATA_DIR, "nutrition_articles.zip")
        
        logger.info("Mencoba mengunduh dataset artikel eksternal...")
        success = download_with_progress(EXTERNAL_ARTICLES_URL, EXTERNAL_ZIP)
        
        if success:
            try:
                import zipfile
                # Ekstrak dataset
                with zipfile.ZipFile(EXTERNAL_ZIP, 'r') as zip_ref:
                    zip_ref.extractall(ARTICLES_DATA_DIR)
                
                # Bersihkan
                os.remove(EXTERNAL_ZIP)
                
                logger.info("Dataset artikel eksternal berhasil diunduh dan diekstrak.")
                return True
            except Exception as e:
                logger.error(f"Gagal mengekstrak dataset artikel eksternal: {e}")
                success = False
        else:
            logger.warning("Gagal mengunduh dataset artikel eksternal. Akan membuat dataset sintetis.")
    
    # Buat dataset sintetis jika diminta atau jika unduhan eksternal gagal
    if create_synthetic:
        try:
            # Definisikan kategori artikel
            categories = ["Gizi Seimbang", "Alergi Makanan", "Perkembangan Anak", "Pencegahan Stunting", "Nutrisi Ibu Hamil"]
            
            # Buat dictionary untuk menyimpan artikel berdasarkan kategori
            articles_data = {category: [] for category in categories}
            
            # Artikel Gizi Seimbang
            articles_data["Gizi Seimbang"] = [
                {
                    "title": "Pentingnya Gizi Seimbang untuk Anak",
                    "summary": "Makanan bergizi sangat penting untuk tumbuh kembang optimal anak.",
                    "content": "Gizi seimbang sangat penting untuk pertumbuhan dan perkembangan optimal anak. "
                              "Pastikan menu harian anak mengandung karbohidrat, protein, lemak sehat, vitamin, "
                              "dan mineral dalam proporsi yang tepat. Contoh menu seimbang adalah nasi, ayam, "
                              "sayuran hijau, buah, dan susu. Perhatikan juga porsi makan yang sesuai dengan usia anak.",
                    "keywords": ["gizi", "nutrisi", "seimbang", "makanan", "vitamin", "mineral"],
                    "age_relevance": [1, 12],  # Relevan untuk usia 1-12 tahun
                    "gender_relevance": "both"  # Relevan untuk kedua gender
                },
                {
                    "title": "Sumber Protein Terbaik untuk Anak",
                    "summary": "Protein adalah nutrisi penting untuk membangun otot dan jaringan tubuh anak.",
                    "content": "Protein adalah nutrisi penting untuk membangun otot dan jaringan tubuh anak. "
                              "Sumber protein berkualitas termasuk telur, daging tanpa lemak, ikan, susu, yogurt, "
                              "kacang-kacangan, dan tahu/tempe. Pastikan anak mendapatkan asupan protein yang cukup "
                              "setiap hari untuk mendukung pertumbuhan optimal.",
                    "keywords": ["protein", "otot", "jaringan", "telur", "daging", "ikan", "kacang"],
                    "age_relevance": [1, 18],  # Relevan untuk usia 1-18 tahun
                    "gender_relevance": "both"  # Relevan untuk kedua gender
                },
                {
                    "title": "Pentingnya Sayuran Hijau untuk Tumbuh Kembang",
                    "summary": "Sayuran hijau mengandung vitamin dan mineral penting untuk anak.",
                    "content": "Sayuran hijau kaya akan vitamin A, C, K, folat, dan mineral seperti zat besi dan kalsium. "
                              "Nutrisi ini sangat penting untuk perkembangan otak, pertumbuhan tulang, penglihatan, "
                              "dan kekebalan tubuh anak. Cobalah mencampurkan sayuran hijau dalam berbagai hidangan "
                              "seperti smoothie, omelet, atau pasta untuk meningkatkan asupan sayur pada anak.",
                    "keywords": ["sayuran", "hijau", "vitamin", "mineral", "zat besi", "kalsium"],
                    "age_relevance": [2, 12],  # Relevan untuk usia 2-12 tahun
                    "gender_relevance": "both"  # Relevan untuk kedua gender
                }
            ]
            
            # Artikel Alergi Makanan
            articles_data["Alergi Makanan"] = [
                {
                    "title": "Mengenali Tanda Alergi Makanan pada Anak",
                    "summary": "Alergi makanan dapat menyebabkan berbagai gejala, dari ringan hingga berat.",
                    "content": "Alergi makanan dapat menyebabkan berbagai gejala, dari ringan hingga berat. "
                              "Gejala umum meliputi ruam kulit, gatal-gatal, bengkak pada bibir atau lidah, "
                              "masalah pencernaan seperti mual, muntah, atau diare, serta kesulitan bernapas "
                              "dalam kasus yang parah. Jika anak menunjukkan gejala ini setelah mengonsumsi "
                              "makanan tertentu, segera konsultasikan dengan dokter.",
                    "keywords": ["alergi", "gejala", "ruam", "bengkak", "mual", "diare"],
                    "age_relevance": [0, 12],  # Relevan untuk usia 0-12 tahun
                    "gender_relevance": "both"  # Relevan untuk kedua gender
                },
                {
                    "title": "Makanan Penyebab Alergi Paling Umum",
                    "summary": "Beberapa makanan yang paling sering menyebabkan alergi pada anak.",
                    "content": "Beberapa makanan yang paling sering menyebabkan alergi pada anak adalah susu sapi, "
                              "telur, kacang tanah, kacang pohon, gandum (gluten), ikan, dan kerang. "
                              "Penting untuk memperkenalkan makanan ini secara bertahap dan mengamati reaksi anak. "
                              "Jika ada riwayat alergi dalam keluarga, diskusikan dengan dokter tentang strategi "
                              "pengenalan makanan yang tepat.",
                    "keywords": ["alergi", "susu", "telur", "kacang", "gluten", "ikan", "kerang"],
                    "age_relevance": [0, 5],  # Relevan untuk usia 0-5 tahun
                    "gender_relevance": "both"  # Relevan untuk kedua gender
                },
                {
                    "title": "Cara Memasak untuk Anak dengan Alergi Makanan",
                    "summary": "Tips dan trik menyiapkan makanan aman untuk anak dengan alergi makanan.",
                    "content": "Memasak untuk anak dengan alergi makanan membutuhkan perhatian ekstra. "
                              "Selalu baca label bahan dengan teliti, hindari kontaminasi silang dengan "
                              "membersihkan alat masak secara menyeluruh, ketahui bahan pengganti yang aman "
                              "(seperti tepung jagung untuk mengganti tepung terigu), dan selalu punya "
                              "rencana cadangan jika terjadi reaksi alergi. Konsultasikan dengan ahli gizi "
                              "untuk memastikan anak tetap mendapatkan nutrisi lengkap meski ada pembatasan makanan.",
                    "keywords": ["alergi", "memasak", "aman", "bahan pengganti", "nutrisi"],
                    "age_relevance": [1, 12],  # Relevan untuk usia 1-12 tahun
                    "gender_relevance": "both"  # Relevan untuk kedua gender
                }
            ]
            
            # Artikel Perkembangan Anak
            articles_data["Perkembangan Anak"] = [
                {
                    "title": "Milestone Perkembangan Anak 0-5 Tahun",
                    "summary": "Pahami tahapan perkembangan anak dari bayi hingga prasekolah.",
                    "content": "Setiap anak berkembang dengan kecepatan berbeda, namun ada milestone umum yang perlu diperhatikan. "
                              "Bayi 0-12 bulan akan belajar mengangkat kepala, berguling, duduk, merangkak, dan mungkin berjalan. "
                              "Anak 1-3 tahun mengembangkan keterampilan bahasa dan kemandirian. Anak 3-5 tahun mengembangkan "
                              "keterampilan sosial dan kognitif yang lebih kompleks. Pantau perkembangan anak dan konsultasikan "
                              "dengan dokter jika ada kekhawatiran.",
                    "keywords": ["milestone", "perkembangan", "motorik", "kognitif", "balita"],
                    "age_relevance": [0, 5],  # Relevan untuk usia 0-5 tahun
                    "gender_relevance": "both"  # Relevan untuk kedua gender
                },
                {
                    "title": "Stimulasi Perkembangan Otak Anak",
                    "summary": "Aktivitas untuk mendukung perkembangan otak optimal pada anak.",
                    "content": "Otak anak berkembang pesat dalam 5 tahun pertama kehidupan. Stimulasi yang tepat dapat mendukung "
                              "perkembangan ini, termasuk berbicara dan membacakan buku sejak bayi, bermain yang melibatkan "
                              "sensorik dan motorik, serta aktivitas yang membangun keterampilan berpikir. Nutrisi yang tepat "
                              "seperti asam lemak omega-3 juga penting untuk perkembangan otak optimal.",
                    "keywords": ["otak", "stimulasi", "kognitif", "membaca", "bermain", "omega-3"],
                    "age_relevance": [0, 5],  # Relevan untuk usia 0-5 tahun
                    "gender_relevance": "both"  # Relevan untuk kedua gender
                },
                {
                    "title": "Nutrisi untuk Mendukung Perkembangan Kognitif",
                    "summary": "Makanan yang membantu perkembangan otak dan keterampilan kognitif anak.",
                    "content": "Perkembangan kognitif anak sangat dipengaruhi oleh asupan nutrisi tertentu. "
                              "Makanan kaya omega-3 seperti ikan, telur dan kenari mendukung perkembangan otak. "
                              "Zat besi dari daging merah, hati, dan sayuran hijau mendukung fungsi kognitif. "
                              "Choline dari telur dan kacang-kacangan membantu pembentukan memori. "
                              "Zinc dari daging, kacang, dan biji-bijian mendukung perhatian dan pembelajaran. "
                              "Pastikan anak mendapatkan nutrisi seimbang ini untuk perkembangan kognitif optimal.",
                    "keywords": ["kognitif", "otak", "omega-3", "zat besi", "choline", "zinc"],
                    "age_relevance": [1, 12],  # Relevan untuk usia 1-12 tahun
                    "gender_relevance": "both"  # Relevan untuk kedua gender
                }
            ]
            
            # Artikel Pencegahan Stunting
            articles_data["Pencegahan Stunting"] = [
                {
                    "title": "Cara Efektif Mencegah Stunting",
                    "summary": "Panduan lengkap mengenali tanda stunting dan cara pencegahannya.",
                    "content": "Stunting dapat dicegah melalui nutrisi yang tepat selama 1000 hari pertama kehidupan, "
                              "mulai dari kehamilan hingga anak berusia 2 tahun. Pastikan ibu hamil mendapatkan nutrisi "
                              "yang cukup, lakukan ASI eksklusif selama 6 bulan pertama, dan berikan MPASI yang bergizi "
                              "seimbang setelahnya. Pantau pertumbuhan anak secara teratur dan pastikan sanitasi yang baik "
                              "untuk mencegah infeksi berulang.",
                    "keywords": ["stunting", "pencegahan", "1000 hari", "ASI", "MPASI", "pertumbuhan"],
                    "age_relevance": [0, 2],  # Relevan untuk usia 0-2 tahun
                    "gender_relevance": "both"  # Relevan untuk kedua gender
                },
                {
                    "title": "Menu Harian Anti-Stunting untuk Balita",
                    "summary": "Contoh menu seimbang untuk mendukung pertumbuhan optimal balita.",
                    "content": "Menu anti-stunting untuk balita harus kaya protein, zat besi, zinc, kalsium, dan vitamin A. "
                              "Contoh menu seimbang meliputi bubur nasi dengan ayam cincang dan sayuran, telur dadar dengan "
                              "sayuran, sup ikan dengan wortel dan bayam, serta buah-buahan segar. Variasikan menu untuk "
                              "memastikan anak mendapatkan beragam nutrisi. Tambahkan sumber protein nabati seperti tempe "
                              "dan tahu untuk menu seimbang.",
                    "keywords": ["menu", "balita", "stunting", "protein", "zat besi", "zinc", "kalsium", "vitamin A"],
                    "age_relevance": [1, 5],  # Relevan untuk usia 1-5 tahun
                    "gender_relevance": "both"  # Relevan untuk kedua gender
                },
                {
                    "title": "Peran Mikronutrien dalam Pencegahan Stunting",
                    "summary": "Vitamin dan mineral penting untuk mencegah stunting pada anak.",
                    "content": "Mikronutrien sangat penting untuk mencegah stunting pada anak. "
                              "Vitamin A mendukung pertumbuhan sel dan sistem kekebalan tubuh. "
                              "Vitamin D membantu penyerapan kalsium untuk pertumbuhan tulang. "
                              "Zat besi mencegah anemia dan mendukung perkembangan otak. "
                              "Zinc berperan dalam produksi hormon pertumbuhan dan kekebalan tubuh. "
                              "Kalsium dan fosfor mendukung pembentukan tulang dan gigi yang kuat. "
                              "Pastikan anak mendapatkan mikronutrien ini dari makanan atau suplemen jika diperlukan.",
                    "keywords": ["mikronutrien", "vitamin", "mineral", "stunting", "pertumbuhan"],
                    "age_relevance": [0, 5],  # Relevan untuk usia 0-5 tahun
                    "gender_relevance": "both"  # Relevan untuk kedua gender
                }
            ]
            
            # Artikel Nutrisi Ibu Hamil
            articles_data["Nutrisi Ibu Hamil"] = [
                {
                    "title": "Nutrisi Penting untuk Ibu Hamil",
                    "summary": "Panduan lengkap nutrisi selama masa kehamilan untuk ibu dan janin sehat.",
                    "content": "Nutrisi yang tepat selama kehamilan sangat penting untuk kesehatan ibu dan perkembangan janin. "
                              "Fokus pada asupan cukup asam folat, zat besi, kalsium, protein, dan omega-3. "
                              "Asam folat membantu mencegah cacat tabung saraf, zat besi mencegah anemia, "
                              "kalsium mendukung perkembangan tulang janin, protein mendukung pertumbuhan jaringan, "
                              "dan omega-3 mendukung perkembangan otak janin. Konsultasikan dengan dokter atau ahli gizi "
                              "untuk panduan nutrisi yang disesuaikan dengan kebutuhan individual.",
                    "keywords": ["kehamilan", "nutrisi", "asam folat", "zat besi", "kalsium", "protein", "omega-3"],
                    "age_relevance": [20, 45],  # Relevan untuk usia 20-45 tahun
                    "gender_relevance": "female"  # Relevan untuk perempuan
                },
                {
                    "title": "Makanan yang Harus Dihindari Selama Kehamilan",
                    "summary": "Daftar makanan yang perlu dihindari untuk kehamilan yang aman dan sehat.",
                    "content": "Beberapa makanan perlu dihindari selama kehamilan untuk mengurangi risiko infeksi dan komplikasi. "
                              "Hindari daging, telur, dan seafood mentah atau setengah matang untuk mencegah infeksi seperti salmonella atau listeria. "
                              "Batasi konsumsi ikan yang tinggi merkuri seperti tuna dan makerel. "
                              "Hindari susu dan produk susu yang tidak dipasteurisasi. "
                              "Batasi kafein dan hindari alkohol sepenuhnya. "
                              "Juga hindari makanan olahan yang tinggi gula dan lemak jenuh.",
                    "keywords": ["kehamilan", "hindari", "mentah", "merkuri", "pasteurisasi", "kafein", "alkohol"],
                    "age_relevance": [20, 45],  # Relevan untuk usia 20-45 tahun
                    "gender_relevance": "female"  # Relevan untuk perempuan
                },
                {
                    "title": "Menu Harian Seimbang untuk Ibu Hamil",
                    "summary": "Contoh menu harian yang memenuhi kebutuhan nutrisi selama kehamilan.",
                    "content": "Menu harian seimbang untuk ibu hamil harus mencakup berbagai kelompok makanan. "
                              "Sarapan: oatmeal dengan susu, buah-buahan, dan kenari. "
                              "Snack pagi: yogurt dengan madu dan pisang. "
                              "Makan siang: nasi merah, ayam panggang, sayuran hijau, dan tahu. "
                              "Snack sore: buah-buahan segar dan kacang-kacangan. "
                              "Makan malam: ikan panggang, kentang, brokoli, dan telur rebus. "
                              "Sebelum tidur: segelas susu hangat. "
                              "Pastikan juga cukup minum air putih sepanjang hari.",
                    "keywords": ["kehamilan", "menu", "seimbang", "nutrisi", "makanan"],
                    "age_relevance": [20, 45],  # Relevan untuk usia 20-45 tahun
                    "gender_relevance": "female"  # Relevan untuk perempuan
                }
            ]
            
            # Simpan artikel sebagai JSON
            with open(articles_json_path, 'w', encoding='utf-8') as f:
                json.dump(articles_data, f, indent=2, ensure_ascii=False)
            
            # Buat dataset tanya jawab nutrisi
            qa_data = []
            for category, articles in articles_data.items():
                for article in articles:
                    # Buat 2-3 pertanyaan per artikel
                    for keyword in article["keywords"][:3]:
                        question = f"Apa pentingnya {keyword} untuk anak?"
                        qa_data.append({
                            "question": question,
                            "category": category,
                            "article_title": article["title"],
                            "age_relevance": article.get("age_relevance", [0, 18]),
                            "gender_relevance": article.get("gender_relevance", "both")
                        })
                        
                        # Tambahkan variasi pertanyaan
                        if category == "Gizi Seimbang":
                            qa_data.append({
                                "question": f"Makanan apa yang mengandung {keyword} tinggi?",
                                "category": category,
                                "article_title": article["title"],
                                "age_relevance": article.get("age_relevance", [0, 18]),
                                "gender_relevance": article.get("gender_relevance", "both")
                            })
                        elif category == "Alergi Makanan":
                            qa_data.append({
                                "question": f"Bagaimana cara mengetahui anak alergi terhadap {keyword}?",
                                "category": category,
                                "article_title": article["title"],
                                "age_relevance": article.get("age_relevance", [0, 18]),
                                "gender_relevance": article.get("gender_relevance", "both")
                            })
                        elif category == "Perkembangan Anak":
                            qa_data.append({
                                "question": f"Kapan anak mulai bisa {keyword}?",
                                "category": category,
                                "article_title": article["title"],
                                "age_relevance": article.get("age_relevance", [0, 18]),
                                "gender_relevance": article.get("gender_relevance", "both")
                            })
                        elif category == "Pencegahan Stunting":
                            qa_data.append({
                                "question": f"Apakah {keyword} dapat mencegah stunting?",
                                "category": category,
                                "article_title": article["title"],
                                "age_relevance": article.get("age_relevance", [0, 18]),
                                "gender_relevance": article.get("gender_relevance", "both")
                            })
                        elif category == "Nutrisi Ibu Hamil":
                            qa_data.append({
                                "question": f"Berapa kebutuhan {keyword} untuk ibu hamil?",
                                "category": category,
                                "article_title": article["title"],
                                "age_relevance": article.get("age_relevance", [20, 45]),
                                "gender_relevance": "female"
                            })
            
            # Konversi ke DataFrame dan simpan sebagai CSV
            qa_df = pd.DataFrame(qa_data)
            qa_df.to_csv(articles_csv_path, index=False)
            
            logger.info(f"Berhasil membuat dataset artikel sintetis dengan {len(qa_data)} pasangan Q&A dari {len(categories)} kategori.")
            return True
        
        except Exception as e:
            logger.error(f"Gagal membuat dataset artikel: {e}")
            return False
    
    return success

# Setup dataset (default: buat dataset sintetis, tanpa mengunduh eksternal)
CREATE_SYNTHETIC = True
DOWNLOAD_EXTERNAL = False  # Ubah menjadi True untuk mencoba mengunduh dataset eksternal
success = setup_articles_dataset(CREATE_SYNTHETIC, DOWNLOAD_EXTERNAL)

if success:
    print("✅ Dataset artikel nutrisi siap digunakan!")
    
    # Tampilkan informasi dataset
    try:
        with open(os.path.join(ARTICLES_DATA_DIR, "articles_data.json"), 'r') as f:
            articles_data = json.load(f)
        
        qa_df = pd.read_csv(os.path.join(ARTICLES_DATA_DIR, "nutrition_qa.csv"))
        
        total_articles = sum(len(articles) for articles in articles_data.values())
        categories = list(articles_data.keys())
        
        print(f"   Total artikel: {total_articles} dalam {len(categories)} kategori")
        for category, articles in articles_data.items():
            print(f"   - {category}: {len(articles)} artikel")
        
        print(f"   Total pertanyaan Q&A: {len(qa_df)}")
    except:
        print("   Tidak dapat membaca informasi dataset, tetapi file telah dibuat.")
else:
    print("❌ Gagal menyiapkan dataset artikel. Periksa log untuk detail.")
    print("   Akan mencoba melanjutkan dengan data minimal.")

### 2.2 Struktur File/Folder yang Diharapkan

Dataset artikel nutrisi disimpan dengan struktur file sebagai berikut:

```
NutriGenius/
├── data/
│   ├── raw/
│   │   └── articles/               # Dataset artikel nutrisi disimpan di sini
│   │       ├── articles_data.json  # Data artikel lengkap
│   │       └── nutrition_qa.csv    # Dataset tanya jawab nutrisi
│   └── processed/
│       └── articles/               # Dataset yang telah diproses
│           ├── article_vectors.pkl # Vektor artikel untuk rekomendasi
│           └── tfidf_model.pkl     # Model TF-IDF terlatih
```

### 2.3 Verifikasi Dataset

Mari kita periksa apakah dataset sudah tersedia dan valid:

In [ ]:
def verify_articles_dataset():
    """Verifikasi dataset artikel nutrisi"""
    # Cek apakah file-file utama ada
    articles_json_path = os.path.join(ARTICLES_DATA_DIR, "articles_data.json")
    articles_csv_path = os.path.join(ARTICLES_DATA_DIR, "nutrition_qa.csv")
    
    if not os.path.exists(articles_json_path):
        logger.error(f"File dataset artikel tidak ditemukan: {articles_json_path}")
        return False
    
    if not os.path.exists(articles_csv_path):
        logger.error(f"File dataset Q&A tidak ditemukan: {articles_csv_path}")
        return False
    
    # Cek apakah file JSON dapat dibuka dan berisi data
    try:
        with open(articles_json_path, 'r') as f:
            articles_data = json.load(f)
        
        if not articles_data or not isinstance(articles_data, dict) or len(articles_data) == 0:
            logger.error("File artikel JSON kosong atau format tidak valid")
            return False
        
        # Cek minimal satu kategori dan artikel
        for category, articles in articles_data.items():
            if len(articles) > 0:
                logger.info(f"Dataset artikel valid dengan {len(articles)} artikel di kategori {category}")
                # Verifikasi data yang diperlukan ada di artikel pertama
                sample_article = articles[0]
                required_fields = ["title", "content", "keywords"]
                if all(field in sample_article for field in required_fields):
                    break
                else:
                    logger.error(f"Format artikel tidak valid, berisi {sample_article.keys()}")
                    return False
        else:
            logger.error("Tidak ada artikel valid dalam dataset")
            return False
    except Exception as e:
        logger.error(f"Gagal membuka atau memproses file JSON: {e}")
        return False
    
    # Cek apakah file CSV dapat dibuka dan berisi data
    try:
        qa_df = pd.read_csv(articles_csv_path)
        if len(qa_df) == 0:
            logger.error("File CSV Q&A kosong")
            return False
        
        required_columns = ["question", "category", "article_title"]
        if not all(col in qa_df.columns for col in required_columns):
            logger.error(f"Format CSV tidak valid, kolom yang ada: {qa_df.columns.tolist()}")
            return False
            
        logger.info(f"Dataset Q&A valid dengan {len(qa_df)} pertanyaan")
    except Exception as e:
        logger.error(f"Gagal membuka atau memproses file CSV: {e}")
        return False
    
    return True

# Verifikasi dataset
is_valid = verify_articles_dataset()
if not is_valid:
    print("⚠️ Dataset artikel tidak valid atau tidak lengkap.")
    print("   Silakan unduh dataset secara manual atau jalankan kode setup_articles_dataset() lagi.")
    
    # Fallback: buat dataset minimal jika perlu
    if not os.path.exists(os.path.join(ARTICLES_DATA_DIR, "articles_data.json")):
        print("   Membuat dataset artikel minimal...")
        # Buat struktur minimal
        minimal_articles = {
            "Gizi Seimbang": [
                {
                    "title": "Pentingnya Gizi Seimbang",
                    "summary": "Makanan bergizi sangat penting untuk tumbuh kembang.",
                    "content": "Gizi seimbang sangat penting untuk pertumbuhan dan perkembangan optimal.",
                    "keywords": ["gizi", "nutrisi", "seimbang"]
                }
            ]
        }
        
        with open(os.path.join(ARTICLES_DATA_DIR, "articles_data.json"), 'w') as f:
            json.dump(minimal_articles, f, indent=2)
        
        minimal_qa = pd.DataFrame([
            {"question": "Apa itu gizi seimbang?", "category": "Gizi Seimbang", "article_title": "Pentingnya Gizi Seimbang"}
        ])
        minimal_qa.to_csv(os.path.join(ARTICLES_DATA_DIR, "nutrition_qa.csv"), index=False)
        
        print("   Dataset artikel minimal telah dibuat. Performa rekomendasi akan terbatas.")

## 3. Eksplorasi Dataset

Setelah kita menyiapkan dataset, mari kita melakukan eksplorasi untuk memahami lebih baik tentang data artikel nutrisi yang kita miliki.

In [ ]:
# Load dataset yang telah dibuat atau diunduh
try:
    # Load data artikel
    with open(os.path.join(ARTICLES_DATA_DIR, "articles_data.json"), 'r', encoding='utf-8') as f:
        articles_data = json.load(f)
    
    # Load data Q&A
    qa_df = pd.read_csv(os.path.join(ARTICLES_DATA_DIR, "nutrition_qa.csv"))
    
    print("Dataset berhasil dimuat!")
except Exception as e:
    logger.error(f"Gagal memuat dataset: {e}")
    print("❌ Gagal memuat dataset. Pastikan dataset telah dibuat dengan menjalankan setup_articles_dataset().")
    # Definisikan variabel kosong untuk menghindari error
    articles_data = {}
    qa_df = pd.DataFrame()

### 3.1 Eksplorasi Dataset Artikel

Mari kita periksa struktur dan distribusi dataset artikel:

In [ ]:
# Mengkonversi data artikel ke format DataFrame untuk memudahkan analisis
article_records = []

for category, articles in articles_data.items():
    for article in articles:
        article_record = article.copy()
        article_record['category'] = category
        article_records.append(article_record)

articles_df = pd.DataFrame(article_records)

# Tampilkan informasi dataset
print(f"Jumlah total artikel: {len(articles_df)}")
print(f"Jumlah kategori: {articles_df['category'].nunique()}")
print("\nDistribusi artikel berdasarkan kategori:")
category_counts = articles_df['category'].value_counts()
for category, count in category_counts.items():
    print(f"- {category}: {count} artikel")

# Tampilkan sampel artikel
print("\nContoh artikel:")
sample_idx = np.random.randint(0, len(articles_df))
sample_article = articles_df.iloc[sample_idx]
print(f"Judul: {sample_article['title']}")
print(f"Kategori: {sample_article['category']}")
print(f"Ringkasan: {sample_article['summary']}")
print(f"Kata kunci: {', '.join(sample_article['keywords'])}")
if 'age_relevance' in sample_article:
    print(f"Rentang usia relevan: {sample_article['age_relevance']}")
if 'gender_relevance' in sample_article:
    print(f"Gender relevan: {sample_article['gender_relevance']}")

In [ ]:
# Visualisasi distribusi kategori artikel
plt.figure(figsize=(10, 6))
sns.countplot(y='category', data=articles_df, order=category_counts.index)
plt.title('Distribusi Artikel Berdasarkan Kategori')
plt.xlabel('Jumlah Artikel')
plt.ylabel('Kategori')
plt.tight_layout()
plt.show()

# Visualisasi distribusi panjang artikel
articles_df['content_length'] = articles_df['content'].apply(len)
articles_df['title_length'] = articles_df['title'].apply(len)

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.histplot(articles_df['content_length'], kde=True)
plt.title('Distribusi Panjang Konten Artikel')
plt.xlabel('Jumlah Karakter')
plt.ylabel('Frekuensi')

plt.subplot(1, 2, 2)
sns.boxplot(x='category', y='content_length', data=articles_df)
plt.title('Panjang Konten berdasarkan Kategori')
plt.xlabel('Kategori')
plt.ylabel('Jumlah Karakter')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### 3.2 Eksplorasi Karakteristik Demografis

Mari kita eksplorasi karakteristik demografis (usia dan gender) dari artikel-artikel tersebut:

In [ ]:
# Analisis distribusi rentang usia
if 'age_relevance' in articles_df.columns:
    # Ekstrak usia minimal dan maksimal
    articles_df['min_age'] = articles_df['age_relevance'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
    articles_df['max_age'] = articles_df['age_relevance'].apply(lambda x: x[1] if isinstance(x, list) and len(x) > 1 else None)
    
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    sns.histplot(articles_df['min_age'].dropna(), kde=True, bins=10)
    plt.title('Distribusi Usia Minimal Relevan')
    plt.xlabel('Usia (Tahun)')
    plt.ylabel('Frekuensi')
    
    plt.subplot(1, 2, 2)
    sns.histplot(articles_df['max_age'].dropna(), kde=True, bins=10)
    plt.title('Distribusi Usia Maksimal Relevan')
    plt.xlabel('Usia (Tahun)')
    plt.ylabel('Frekuensi')
    plt.tight_layout()
    plt.show()
    
    # Visualisasi distribusi rentang usia berdasarkan kategori
    plt.figure(figsize=(12, 6))
    sns.boxplot(x='category', y='min_age', data=articles_df)
    plt.title('Rentang Usia Minimal berdasarkan Kategori')
    plt.xlabel('Kategori')
    plt.ylabel('Usia (Tahun)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Analisis distribusi gender
if 'gender_relevance' in articles_df.columns:
    gender_counts = articles_df['gender_relevance'].value_counts()
    plt.figure(figsize=(8, 6))
    sns.countplot(y='gender_relevance', data=articles_df)
    plt.title('Distribusi Artikel berdasarkan Gender Relevan')
    plt.xlabel('Jumlah Artikel')
    plt.ylabel('Gender Relevan')
    
    # Tambahkan anotasi persentase
    total = len(articles_df)
    for i, count in enumerate(gender_counts):
        plt.text(count + 1, i, f'{count} ({count/total:.1%})')
        
    plt.tight_layout()
    plt.show()
    
    # Visualisasi distribusi gender berdasarkan kategori
    plt.figure(figsize=(12, 6))
    gender_category = pd.crosstab(articles_df['category'], articles_df['gender_relevance'])
    gender_category.plot(kind='bar', stacked=True)
    plt.title('Distribusi Gender Relevan berdasarkan Kategori')
    plt.xlabel('Kategori')
    plt.ylabel('Jumlah Artikel')
    plt.xticks(rotation=45)
    plt.legend(title='Gender Relevan')
    plt.tight_layout()
    plt.show()

### 3.3 Eksplorasi Dataset Tanya Jawab (Q&A)

Sekarang mari kita eksplorasi dataset tanya jawab nutrisi:

In [ ]:
# Tampilkan informasi dataset Q&A
if not qa_df.empty:
    print(f"Jumlah total pertanyaan: {len(qa_df)}")
    print(f"Jumlah kategori: {qa_df['category'].nunique()}")
    print("\nDistribusi pertanyaan berdasarkan kategori:")
    qa_category_counts = qa_df['category'].value_counts()
    for category, count in qa_category_counts.items():
        print(f"- {category}: {count} pertanyaan")
    
    # Tampilkan beberapa sampel pertanyaan
    print("\nContoh pertanyaan:")
    sample_questions = qa_df.sample(min(5, len(qa_df)))
    for i, (_, row) in enumerate(sample_questions.iterrows()):
        print(f"{i+1}. {row['question']}")
        print(f"   Kategori: {row['category']}")
        print(f"   Artikel terkait: {row['article_title']}")
        if 'age_relevance' in row and not pd.isna(row['age_relevance']):
            print(f"   Rentang usia relevan: {row['age_relevance']}")
        if 'gender_relevance' in row and not pd.isna(row['gender_relevance']):
            print(f"   Gender relevan: {row['gender_relevance']}")
        print()
    
    # Visualisasi distribusi pertanyaan berdasarkan kategori
    plt.figure(figsize=(10, 6))
    sns.countplot(y='category', data=qa_df, order=qa_category_counts.index)
    plt.title('Distribusi Pertanyaan Berdasarkan Kategori')
    plt.xlabel('Jumlah Pertanyaan')
    plt.ylabel('Kategori')
    plt.tight_layout()
    plt.show()
    
    # Analisis panjang pertanyaan
    qa_df['question_length'] = qa_df['question'].apply(len)
    
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    sns.histplot(qa_df['question_length'], kde=True)
    plt.title('Distribusi Panjang Pertanyaan')
    plt.xlabel('Jumlah Karakter')
    plt.ylabel('Frekuensi')
    
    plt.subplot(1, 2, 2)
    sns.boxplot(x='category', y='question_length', data=qa_df)
    plt.title('Panjang Pertanyaan berdasarkan Kategori')
    plt.xlabel('Kategori')
    plt.ylabel('Jumlah Karakter')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("Dataset Q&A kosong.")

## 4. Analisis Teks

Sekarang mari kita lakukan analisis teks pada artikel dan dataset tanya jawab untuk memahami konten dan kata kunci.

### 4.1 Text Preprocessing

Pertama, kita akan melakukan preprocessing pada teks artikel:

In [ ]:
# Fungsi untuk preprocessing teks
def preprocess_text(text):
    # Konversi ke lowercase
    text = text.lower()
    
    # Hapus karakter khusus dan angka
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    
    # Tokenisasi
    tokens = word_tokenize(text)
    
    # Hapus stopwords
    stop_words = set(stopwords.words('indonesian') + stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Gabungkan kembali
    cleaned_text = ' '.join(tokens)
    
    return cleaned_text

# Terapkan preprocessing pada konten artikel
try:
    articles_df['preprocessed_content'] = articles_df['content'].apply(preprocess_text)
    articles_df['preprocessed_title'] = articles_df['title'].apply(preprocess_text)
    
    # Terapkan preprocessing pada pertanyaan
    if not qa_df.empty:
        qa_df['preprocessed_question'] = qa_df['question'].apply(preprocess_text)
    
    print("Text preprocessing selesai!")
except Exception as e:
    logger.error(f"Gagal melakukan preprocessing: {e}")
    print("❌ Text preprocessing gagal.")

### 4.2 Analisis Kata Kunci

Mari kita analisis kata kunci dari setiap kategori artikel:

In [ ]:
# Fungsi untuk mendapatkan kata-kata yang sering muncul
def get_top_words(texts, n=20):
    from collections import Counter
    
    # Gabungkan semua teks
    all_words = ' '.join(texts).split()
    
    # Hitung frekuensi kata
    word_counts = Counter(all_words)
    
    # Ambil n kata teratas
    top_words = word_counts.most_common(n)
    
    return top_words

# Analisis kata-kata yang sering muncul per kategori
for category in articles_df['category'].unique():
    category_articles = articles_df[articles_df['category'] == category]
    top_words = get_top_words(category_articles['preprocessed_content'], n=10)
    
    print(f"\nKata kunci teratas untuk kategori '{category}':")
    for word, count in top_words:
        print(f"- {word}: {count}")
    
    # Visualisasi menggunakan bar chart
    plt.figure(figsize=(10, 5))
    words, counts = zip(*top_words)
    sns.barplot(x=list(counts), y=list(words))
    plt.title(f'10 Kata Teratas - Kategori: {category}')
    plt.xlabel('Frekuensi')
    plt.ylabel('Kata')
    plt.tight_layout()
    plt.show()

### 4.3 Visualisasi Word Cloud

Word cloud adalah cara yang efektif untuk memvisualisasikan kata-kata yang sering muncul dalam teks:

In [ ]:
# Membuat word cloud untuk setiap kategori
for category in articles_df['category'].unique():
    category_articles = articles_df[articles_df['category'] == category]
    text = ' '.join(category_articles['preprocessed_content'])
    
    # Buat word cloud
    wordcloud = WordCloud(width=800, height=400, background_color='white', max_words=100, colormap='viridis').generate(text)
    
    plt.figure(figsize=(12, 6))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(f'Word Cloud untuk Kategori: {category}')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Word cloud untuk semua artikel
all_text = ' '.join(articles_df['preprocessed_content'])
all_wordcloud = WordCloud(width=800, height=400, background_color='white', max_words=100, colormap='plasma').generate(all_text)

plt.figure(figsize=(12, 6))
plt.imshow(all_wordcloud, interpolation='bilinear')
plt.title('Word Cloud untuk Semua Artikel')
plt.axis('off')
plt.tight_layout()
plt.show()

### 4.4 Analisis Kata Kunci dari Tanya Jawab

Mari kita analisis kata kunci dari pertanyaan-pertanyaan dalam dataset Q&A:

In [ ]:
if not qa_df.empty:
    # Word cloud untuk pertanyaan
    questions_text = ' '.join(qa_df['preprocessed_question'])
    questions_wordcloud = WordCloud(width=800, height=400, background_color='white', 
                                   max_words=100, colormap='inferno').generate(questions_text)
    
    plt.figure(figsize=(12, 6))
    plt.imshow(questions_wordcloud, interpolation='bilinear')
    plt.title('Word Cloud untuk Pertanyaan Q&A')
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    # Kata-kata yang sering muncul dalam pertanyaan
    top_question_words = get_top_words(qa_df['preprocessed_question'], n=15)
    
    print("\nKata kunci teratas dalam pertanyaan:")
    for word, count in top_question_words:
        print(f"- {word}: {count}")
    
    # Visualisasi menggunakan bar chart
    plt.figure(figsize=(10, 6))
    words, counts = zip(*top_question_words)
    sns.barplot(x=list(counts), y=list(words))
    plt.title('15 Kata Teratas dalam Pertanyaan')
    plt.xlabel('Frekuensi')
    plt.ylabel('Kata')
    plt.tight_layout()
    plt.show()

### 4.5 TF-IDF Analysis

TF-IDF (Term Frequency-Inverse Document Frequency) adalah metode pembobotan kata yang membantu mengidentifikasi kata-kata penting dalam dokumen:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Gunakan TF-IDF Vectorizer untuk mengidentifikasi kata-kata kunci
try:
    # Terapkan TF-IDF pada konten artikel
    tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words=stopwords.words('indonesian') + stopwords.words('english'))
    tfidf_matrix = tfidf_vectorizer.fit_transform(articles_df['preprocessed_content'])
    
    # Dapatkan fitur (kata) dari vectorizer
    feature_names = tfidf_vectorizer.get_feature_names_out()
    
    print(f"Ukuran matriks TF-IDF: {tfidf_matrix.shape}")
    print(f"Jumlah fitur (kata): {len(feature_names)}")
    
    # Lihat beberapa fitur teratas
    print("\nContoh fitur (kata) dari TF-IDF vectorizer:")
    print(", ".join(feature_names[:20]))
    
    # Simpan vectorizer untuk digunakan di notebook model
    import pickle
    tfidf_model_path = os.path.join(PROCESSED_DATA_DIR, "tfidf_model.pkl")
    with open(tfidf_model_path, 'wb') as f:
        pickle.dump(tfidf_vectorizer, f)
    print(f"\nTF-IDF vectorizer disimpan di: {tfidf_model_path}")
    
    # Identifikasi kata-kata penting untuk setiap kategori
    for category in articles_df['category'].unique():
        category_indices = articles_df[articles_df['category'] == category].index
        category_tfidf = tfidf_matrix[category_indices]
        
        # Hitung rata-rata TF-IDF untuk setiap kata dalam kategori
        category_tfidf_mean = category_tfidf.mean(axis=0)
        
        # Konversi ke array
        category_tfidf_mean = category_tfidf_mean.toarray().flatten()
        
        # Dapatkan indeks kata-kata dengan nilai TF-IDF tertinggi
        top_word_indices = category_tfidf_mean.argsort()[-10:][::-1]
        top_words = [(feature_names[i], category_tfidf_mean[i]) for i in top_word_indices]
        
        print(f"\nKata-kata penting untuk kategori '{category}' (berdasarkan TF-IDF):")
        for word, score in top_words:
            print(f"- {word}: {score:.4f}")
except Exception as e:
    logger.error(f"Gagal melakukan analisis TF-IDF: {e}")
    print("❌ Analisis TF-IDF gagal.")

## 5. Visualisasi Data

Sekarang mari kita buat beberapa visualisasi tambahan untuk membantu memahami dataset dengan lebih baik.

### 5.1 Visualisasi Hubungan antar Artikel

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE

try:
    # Hitung similarity antar artikel
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    
    # Terapkan t-SNE untuk visualisasi dalam ruang 2D
    tsne = TSNE(n_components=2, random_state=42, perplexity=min(30, len(articles_df)-1))
    tsne_results = tsne.fit_transform(tfidf_matrix.toarray())
    
    # Tambahkan hasil t-SNE ke DataFrame
    articles_df['tsne_1'] = tsne_results[:, 0]
    articles_df['tsne_2'] = tsne_results[:, 1]
    
    # Visualisasi hasil t-SNE dengan kategori sebagai warna
    plt.figure(figsize=(12, 8))
    categories = articles_df['category'].unique()
    
    for i, category in enumerate(categories):
        category_df = articles_df[articles_df['category'] == category]
        plt.scatter(category_df['tsne_1'], category_df['tsne_2'], label=category, alpha=0.7)
    
    plt.title('Visualisasi t-SNE dari Artikel Berdasarkan Konten (TF-IDF)')
    plt.xlabel('t-SNE Dimensi 1')
    plt.ylabel('t-SNE Dimensi 2')
    plt.legend(title='Kategori')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()
except Exception as e:
    logger.error(f"Gagal melakukan visualisasi t-SNE: {e}")
    print("❌ Visualisasi t-SNE gagal.")

### 5.2 Network Visualization (Artikel yang Terkait)

In [ ]:
try:
    # Membuat network visualization untuk artikel yang terkait
    # Kita hanya tampilkan koneksi dengan similarity di atas threshold
    threshold = 0.3
    
    # Siapkan data untuk visualisasi jaringan
    import networkx as nx
    
    # Buat graph
    G = nx.Graph()
    
    # Tambahkan node untuk setiap artikel
    for i, row in articles_df.iterrows():
        G.add_node(i, title=row['title'], category=row['category'])
    
    # Tambahkan edge untuk artikel yang terkait
    for i in range(len(articles_df)):
        for j in range(i+1, len(articles_df)):
            similarity = cosine_sim[i, j]
            if similarity > threshold:
                G.add_edge(i, j, weight=similarity)
    
    # Visualisasi jaringan
    plt.figure(figsize=(12, 12))
    
    # Position nodes using force-directed layout
    pos = nx.spring_layout(G, seed=42)
    
    # Warna node berdasarkan kategori
    colors = [list(categories).index(articles_df.loc[node, 'category']) for node in G.nodes()]
    
    # Ukuran node berdasarkan degree
    node_size = [300 * (1 + G.degree(node)) for node in G.nodes()]
    
    # Gambar nodes
    nx.draw_networkx_nodes(G, pos, 
                          node_color=colors, 
                          node_size=node_size,
                          alpha=0.8, 
                          cmap=plt.cm.tab10)
    
    # Gambar edges
    edge_weights = [G[u][v]['weight'] * 2 for u, v in G.edges()]
    nx.draw_networkx_edges(G, pos, width=edge_weights, alpha=0.3)
    
    # Tambahkan label pada node dengan degree tertinggi
    top_nodes = sorted(G.degree, key=lambda x: x[1], reverse=True)[:5]
    labels = {node: articles_df.loc[node, 'title'][:20] + '...' for node, degree in top_nodes}
    nx.draw_networkx_labels(G, pos, labels=labels, font_size=10)
    
    # Legenda untuk kategori
    handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=plt.cm.tab10(i), markersize=10) 
              for i in range(len(categories))]
    plt.legend(handles, categories, title='Kategori', loc='upper left', bbox_to_anchor=(1, 1))
    
    plt.title('Jaringan Artikel yang Terkait (Similarity > {:.1f})'.format(threshold))
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    # Tampilkan statistik jaringan
    print(f"Statistik jaringan artikel:")
    print(f"- Jumlah node (artikel): {G.number_of_nodes()}")
    print(f"- Jumlah edge (koneksi): {G.number_of_edges()}")
    print(f"- Jumlah komponen terkoneksi: {nx.number_connected_components(G)}")
    
    # Temukan komunitas
    try:
        from community import community_louvain
        
        # Deteksi komunitas menggunakan algoritma Louvain
        partition = community_louvain.best_partition(G)
        
        # Hitung jumlah komunitas
        communities = set(partition.values())
        print(f"- Jumlah komunitas terdeteksi: {len(communities)}")
        
        # Hitung distribusi artikel per komunitas
        community_counts = pd.Series(partition).value_counts()
        print("\nDistribusi artikel per komunitas:")
        for comm, count in community_counts.items():
            print(f"- Komunitas {comm}: {count} artikel")
    except ImportError:
        print("Package 'community' tidak tersedia. Analisis komunitas tidak dilakukan.")
except Exception as e:
    logger.error(f"Gagal melakukan visualisasi jaringan: {e}")
    print("❌ Visualisasi jaringan gagal.")

### 5.3 Klasifikasi dan Pengelompokan Kata Kunci

In [ ]:
# Menganalisis dan mengelompokkan kata kunci dari artikel
try:
    # Mengumpulkan semua kata kunci
    all_keywords = []
    for keywords in articles_df['keywords']:
        all_keywords.extend(keywords)
    
    # Menghitung frekuensi kata kunci
    keyword_counts = pd.Series(all_keywords).value_counts()
    
    # Menampilkan 20 kata kunci teratas
    print("20 Kata kunci teratas di semua artikel:")
    for keyword, count in keyword_counts.head(20).items():
        print(f"- {keyword}: {count}")
    
    # Visualisasi 20 kata kunci teratas
    plt.figure(figsize=(12, 6))
    keyword_counts.head(20).plot(kind='bar')
    plt.title('20 Kata Kunci Teratas di Semua Artikel')
    plt.xlabel('Kata Kunci')
    plt.ylabel('Frekuensi')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # Membuat matriks co-occurrence untuk kata kunci
    from collections import defaultdict
    
    # Inisialisasi matriks co-occurrence
    co_occurrence = defaultdict(lambda: defaultdict(int))
    
    # Hitung co-occurrence
    for keywords in articles_df['keywords']:
        for i, kw1 in enumerate(keywords):
            for kw2 in keywords[i:]:
                if kw1 != kw2:
                    co_occurrence[kw1][kw2] += 1
                    co_occurrence[kw2][kw1] += 1
    
    # Ambil kata kunci teratas untuk visualisasi
    top_keywords = keyword_counts.head(15).index.tolist()
    
    # Buat matriks co-occurrence untuk kata kunci teratas
    co_matrix = pd.DataFrame(0, index=top_keywords, columns=top_keywords)
    for kw1 in top_keywords:
        for kw2 in top_keywords:
            co_matrix.loc[kw1, kw2] = co_occurrence[kw1][kw2]
    
    # Visualisasi heatmap
    plt.figure(figsize=(12, 10))
    sns.heatmap(co_matrix, annot=True, cmap='YlGnBu', fmt='d')
    plt.title('Matriks Co-occurrence Kata Kunci Teratas')
    plt.tight_layout()
    plt.show()
    
    # Membuat visualisasi jaringan untuk kata kunci yang saling terkait
    G_kw = nx.Graph()
    
    # Tambahkan node untuk setiap kata kunci teratas
    for kw in top_keywords:
        G_kw.add_node(kw, count=keyword_counts[kw])
    
    # Tambahkan edge untuk kata kunci yang co-occur
    for i, kw1 in enumerate(top_keywords):
        for kw2 in top_keywords[i+1:]:
            weight = co_matrix.loc[kw1, kw2]
            if weight > 0:
                G_kw.add_edge(kw1, kw2, weight=weight)
    
    # Visualisasi jaringan kata kunci
    plt.figure(figsize=(12, 12))
    
    # Position nodes using force-directed layout
    pos_kw = nx.spring_layout(G_kw, k=0.3, seed=42)
    
    # Ukuran node berdasarkan frekuensi kata kunci
    node_size = [100 * (1 + keyword_counts[kw]) for kw in G_kw.nodes()]
    
    # Warna node berdasarkan degree
    node_color = [G_kw.degree(kw) for kw in G_kw.nodes()]
    
    # Gambar nodes
    nx.draw_networkx_nodes(G_kw, pos_kw, 
                          node_color=node_color, 
                          node_size=node_size,
                          alpha=0.8, 
                          cmap=plt.cm.viridis)
    
    # Gambar edges
    edge_weights = [G_kw[u][v]['weight'] / 2 for u, v in G_kw.edges()]
    nx.draw_networkx_edges(G_kw, pos_kw, width=edge_weights, alpha=0.5)
    
    # Tambahkan label pada semua node
    labels = {kw: kw for kw in G_kw.nodes()}
    nx.draw_networkx_labels(G_kw, pos_kw, labels=labels, font_size=10)
    
    plt.title('Jaringan Kata Kunci yang Saling Terkait')
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    logger.error(f"Gagal melakukan analisis kata kunci: {e}")
    print("❌ Analisis kata kunci gagal.")

## 6. Kesimpulan

Dalam notebook ini, kita telah melakukan eksplorasi komprehensif terhadap dataset artikel nutrisi untuk sistem rekomendasi pada aplikasi NutriGenius. Berikut adalah ringkasan temuan utama:

### 6.1 Rangkuman Temuan Eksplorasi Data

1. **Struktur Dataset**:
   - Dataset terdiri dari 5 kategori artikel nutrisi, yaitu Gizi Seimbang, Alergi Makanan, Perkembangan Anak, Pencegahan Stunting, dan Nutrisi Ibu Hamil.
   - Setiap kategori memiliki 3 artikel, dengan total 15 artikel dalam dataset.
   - Dataset juga berisi tanya jawab (Q&A) yang terkait dengan artikel-artikel tersebut.

2. **Karakteristik Konten**:
   - Artikel memiliki struktur yang konsisten (judul, ringkasan, konten, kata kunci).
   - Konten artikel berfokus pada topik-topik nutrisi yang relevan dengan aplikasi NutriGenius.
   - Artikel dilengkapi dengan metadata demografis (rentang usia dan gender yang relevan).

3. **Distribusi Demografis**:
   - Mayoritas artikel relevan untuk kedua gender ("both").
   - Kategori "Nutrisi Ibu Hamil" khusus untuk gender perempuan.
   - Rentang usia bervariasi, dengan kategori "Pencegahan Stunting" fokus pada usia 0-5 tahun dan "Nutrisi Ibu Hamil" untuk usia dewasa (20-45 tahun).

4. **Analisis Teks**:
   - Identifikasi kata-kata kunci untuk setiap kategori menggunakan frekuensi kata dan TF-IDF.
   - Visualisasi word cloud menunjukkan fokus untuk setiap kategori (misalnya "stunting", "pertumbuhan" untuk kategori Pencegahan Stunting).
   - Analisis jaringan kata kunci menunjukkan hubungan antar konsep nutrisi.

5. **Hubungan Antar Artikel**:
   - Visualisasi t-SNE menunjukkan pengelompokan artikel berdasarkan konten.
   - Analisis jaringan menunjukkan artikel yang terkait berdasarkan kemiripan konten.
   - Artikel dalam kategori yang sama cenderung memiliki kemiripan konten yang lebih tinggi.

### 6.2 Implikasi untuk Sistem Rekomendasi

Berdasarkan eksplorasi ini, kita dapat mengembangkan sistem rekomendasi dengan fitur-fitur berikut:

1. **Rekomendasi Berbasis Konten**:
   - Menggunakan representasi TF-IDF untuk menghitung kemiripan artikel.
   - Merekomendasikan artikel dengan konten yang mirip dengan yang pernah dibaca pengguna.

2. **Rekomendasi Berbasis Demografi**:
   - Merekomendasikan artikel yang relevan dengan usia dan gender pengguna, yang diperoleh dari komponen deteksi wajah.
   - Contoh: Artikel "Nutrisi Ibu Hamil" untuk pengguna perempuan usia 20-45 tahun.

3. **Rekomendasi Berbasis Konteks**:
   - Merekomendasikan artikel berdasarkan makanan yang terdeteksi oleh komponen deteksi objek makanan.
   - Contoh: Artikel tentang protein ketika pengguna memotret makanan dengan kandungan protein tinggi.

4 **Integrasi dengan Komponen Lain**:
   - Menggunakan hasil dari komponen deteksi wajah untuk menentukan usia dan gender pengguna.
   - Menggunakan hasil dari komponen deteksi makanan untuk konteks rekomendasi.
   - Menggunakan hasil dari komponen pengenalan teks (OCR) untuk mencocokkan informasi nutrisi dengan artikel yang relevan.

### 6.3 Langkah Selanjutnya

Berdasarkan eksplorasi data ini, langkah-langkah selanjutnya dalam pengembangan sistem rekomendasi artikel adalah:

1. **Pengembangan Model**:
   - Mengembangkan model rekomendasi berbasis konten menggunakan representasi TF-IDF.
   - Menerapkan dimensionality reduction (seperti LSA/LDA) untuk meningkatkan kualitas rekomendasi.

2. **Fitur Demografis dan Kontekstual**:
   - Mengimplementasikan fitur untuk memfilter rekomendasi berdasarkan usia dan gender.
   - Mengembangkan pemetaan antara kategori makanan dan kategori artikel yang relevan.

3. **Evaluasi dan Optimasi**:
   - Mengevaluasi kualitas rekomendasi menggunakan metrik seperti precision, recall, dan relevance.
   - Mengoptimalkan sistem berdasarkan feedback pengguna.

4. **Implementasi**:
   - Mengintegrasikan sistem rekomendasi ke dalam aplikasi NutriGenius.
   - Mengembangkan antarmuka pengguna yang intuitif untuk menyajikan rekomendasi artikel.

Eksplorasi dataset ini memberikan fondasi yang kuat untuk mengembangkan sistem rekomendasi artikel yang personal dan kontekstual, yang dapat meningkatkan pengalaman pengguna aplikasi NutriGenius. Dalam notebook model selanjutnya, kita akan melanjutkan dengan implementasi sistem rekomendasi artikel yang personal dan kontekstual.

In [ ]:
# Simpan hasil eksplorasi untuk digunakan di notebook model
try:
    # Simpan DataFrame artikel yang telah dipreprocessing
    articles_df_path = os.path.join(PROCESSED_DATA_DIR, "processed_articles_df.pkl")
    articles_df.to_pickle(articles_df_path)
    
    # Simpan matriks TF-IDF dan cosine similarity
    tfidf_matrix_path = os.path.join(PROCESSED_DATA_DIR, "tfidf_matrix.npz")
    import scipy.sparse as sp
    sp.save_npz(tfidf_matrix_path, tfidf_matrix)
    
    # Simpan matriks cosine similarity jika sudah dihitung
    try:
        cosine_sim_path = os.path.join(PROCESSED_DATA_DIR, "cosine_sim_matrix.npy")
        np.save(cosine_sim_path, cosine_sim)
    except NameError:
        pass
    
    print("\n✅ Hasil eksplorasi telah disimpan dan siap digunakan untuk pemodelan!")
    print(f"   - DataFrame artikel: {articles_df_path}")
    print(f"   - Matriks TF-IDF: {tfidf_matrix_path}")
    
    # Tampilkan statistik akhir
    print("\nStatistik Akhir Dataset Artikel:")
    print(f"   - Jumlah artikel: {len(articles_df)}")
    print(f"   - Jumlah kategori: {articles_df['category'].nunique()}")
    print(f"   - Jumlah kata unik (fitur): {len(feature_names) if 'feature_names' in locals() else 'N/A'}")
    print(f"   - Jumlah pertanyaan Q&A: {len(qa_df)}")
    
except Exception as e:
    logger.error(f"Gagal menyimpan hasil eksplorasi: {e}")
    print("❌ Gagal menyimpan hasil eksplorasi.")

---

Dengan eksplorasi data ini, kita telah memperoleh pemahaman mendalam tentang dataset artikel nutrisi dan karakteristiknya. Informasi ini akan sangat berharga untuk mengembangkan sistem rekomendasi yang efektif dalam aplikasi NutriGenius. Notebook model berikutnya akan memanfaatkan hasil eksplorasi ini untuk membangun sistem rekomendasi artikel yang personal dan kontekstual.